In [265]:
import numpy as np
import pandas as pd
from collections import Counter

In [266]:
def create_data():
    record = [['青年', '否', '否', '一般', '否'],
              ['青年', '否', '否', '好', '否'],
              ['青年', '是', '否', '好', '是'],
              ['青年', '是', '是', '一般', '是'],
              ['青年', '否', '否', '一般', '否'],
              ['中年', '否', '否', '一般', '否'],
              ['中年', '否', '否', '好', '否'],
              ['中年', '是', '是', '好', '是'],
              ['中年', '否', '是', '非常好', '是'],
              ['中年', '否', '是', '非常好', '是'],
              ['老年', '否', '是', '非常好', '是'],
              ['老年', '否', '是', '好', '是'],
              ['老年', '是', '否', '好', '是'],
              ['老年', '是', '否', '非常好', '是'],
              ['老年', '否', '否', '一般', '否'],
              ]
    tags = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和各特征的名称
    return np.array(record), tags

In [267]:
data, labels = create_data()
train_data = pd.DataFrame(data, columns=labels)
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [268]:
def gini_best(arr):
    def gini(y):
        """计算y的基尼指数"""
        cate = Counter(y)
        pro_vector = np.array(list(cate.values())) / len(y)
        res = 1 - pro_vector @ pro_vector

        return res

    if gini(arr[:, -1]) == 0:  # 判断是否为叶结点
        return '叶结点,停止分裂'

    gini_dict = {}
    for i in range(arr.shape[1] - 1):
        counter = Counter(arr[:, i])  # 特征arr[:, i]不同取值及其个数
        key = list(counter.keys())
        counter_pro = dict(zip(counter.keys(), np.array(list(counter.values())) / 15))  # 特征arr[:, i]不同取值及其所占的比例
        if len(key) == 1:
            gi = gini(data[data[:, 1] == key[0]][:, -1])
            gini_dict[i] = [key[0], gi]
        elif len(key) == 2:
            gi = counter_pro[key[0]] * gini(data[data[:, i] == key[0]][:, -1]) +
            counter_pro[key[1]] * gini(data[data[:, i] == key[1]][:, -1])
        gini_dict[i] = [key[0], gi]
    else:
        temp_gini = list()
        for j in key:
            gi = counter_pro[j] * gini(data[data[:, i] == j][:, -1]) +
            (1 - counter_pro[j]) * gini(data[data[:, i] != j][:, -1])  # 将取值为j视为一类,取值不为j视为另一类
        temp_gini.append([j, gi])
    gini_dict[i] = min(temp_gini, key=lambda x: x[1])


return gini_dict  # 返回不同特征的最优划分点和最优划分点对应的基尼指数

In [269]:
first_record = gini_best(data)
first_record

{0: ['老年', 0.43999999999999995],
 1: ['否', 0.31999999999999995],
 2: ['否', 0.26666666666666666],
 3: ['一般', 0.31999999999999984]}

In [270]:
def find_best_split(arr, pro_record):
    """进行一次划分(二叉树)"""
    best_gini = min(list(pro_record.values()), key=lambda x: x[1])
    split_point = best_gini[0]
    split_dim = 0
    for i in pro_record.keys():
        if pro_record[i] == best_gini:
            split_dim = i

    r1 = arr[arr[:, split_dim] == split_point]
    r2 = arr[arr[:, split_dim] != split_point]

    return r1, r2

In [271]:
left, right = find_best_split(data, first_record)  # 第一次划分
print(left)
print(right)

[['青年' '否' '否' '一般' '否']
 ['青年' '否' '否' '好' '否']
 ['青年' '是' '否' '好' '是']
 ['青年' '否' '否' '一般' '否']
 ['中年' '否' '否' '一般' '否']
 ['中年' '否' '否' '好' '否']
 ['老年' '是' '否' '好' '是']
 ['老年' '是' '否' '非常好' '是']
 ['老年' '否' '否' '一般' '否']]
[['青年' '是' '是' '一般' '是']
 ['中年' '是' '是' '好' '是']
 ['中年' '否' '是' '非常好' '是']
 ['中年' '否' '是' '非常好' '是']
 ['老年' '否' '是' '非常好' '是']
 ['老年' '否' '是' '好' '是']]


In [276]:
gini_best(right)

'叶节点,停止分裂'

In [277]:
gini_best(left)

{0: ['青年', 0.43600000000000005],
 1: ['否', 0.192],
 2: ['否', 0.48],
 3: ['一般', 0.31999999999999984]}

In [279]:
left2, right2 = find_best_split(left, gini_best(left))  # 第二次划分
print(left2)
print(right2)

[['青年' '否' '否' '一般' '否']
 ['青年' '否' '否' '好' '否']
 ['青年' '否' '否' '一般' '否']
 ['中年' '否' '否' '一般' '否']
 ['中年' '否' '否' '好' '否']
 ['老年' '否' '否' '一般' '否']]
[['青年' '是' '否' '好' '是']
 ['老年' '是' '否' '好' '是']
 ['老年' '是' '否' '非常好' '是']]
